In [7]:
import datetime as dt
from collections import OrderedDict

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import curio
import curio_http

In [24]:
def price_to_float(price_string):
    return float(price_string.replace('$', ''))
              
def parse_product_info(html):
    """
    Given a response from the Countdown API, parse it, and return in as a dictionary.
    """
    # Parse response
    d = OrderedDict()
    
    soup = BeautifulSoup(html, 'lxml')
    
    s = soup.select('div.product-title > h1')
    d['name'] = list(s[0].stripped_strings)[0]
    
    s = soup.select('p.product-description-text')
    ss = list(s[0].stripped_strings)
    if ss:
        d['description'] = ss[0]
    else:
        d['description'] = None
    

    s = soup.select('span.volume-size')
    ss = list(s[0].stripped_strings)
    if ss:
        d['size'] = ss[0]
    else:
        d['size'] = None

    s1 = soup.select('span.special-price')
    s2 = soup.select('span.club-price-wrapper')
    s3 = soup.select('span.price')
    if s1:
        d['on_sale'] = True
        ss = list(s1[0].stripped_strings)
        d['sale_price'] = price_to_float(ss[0])
        t = soup.select('span.was-price')
        tt = list(t[0].stripped_strings)
        d['price'] = price_to_float(tt[0].replace('was', ''))
    elif s2:
        d['on_sale'] = True
        ss = list(s2[0].stripped_strings)
        d['sale_price'] = price_to_float(ss[0])
        t = soup.select('span.grid-non-club-price')
        tt = list(t[0].stripped_strings)
        d['price'] = price_to_float(tt[0].replace('non club price', ''))
    elif s3:
        d['on_sale'] = False
        d['sale_price'] = None    
        ss = list(s3[0].stripped_strings)
        d['price'] = price_to_float(ss[0])
    
    s = soup.select('div.cup-price')
    ss = list(s[0].stripped_strings)
    if ss:
        d['cup_price'] = ss[0]
    else:
        d['cup_price'] = None
        
    d['datetime'] = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    return d

def collect_product_info_1(stock_codes, as_df=True):
    """
    For each product stock code (string) in the list ``stock_codes``,
    issue a GET request to Countdown at https://shop.countdown.co.nz/Shop/ProductDetails?
    to retrieve product information.
    Send all the requests at once and return the result as a generator of responses.
    """
    url = 'https://shop.countdown.co.nz/Shop/ProductDetails?'
    results = []
    for code in stock_codes:
        r = requests.get(url, params={'stockcode': code})
        if r.status_code == 200:
            info = parse_product_info(r.text)
        results.append(info)
        
    if as_df:
        results = pd.DataFrame(results)
        results['datetime'] = pd.to_datetime(results['datetime'])
        
    return results



In [25]:
# Try async with curio

MAX_CONNECTIONS_PER_HOST = 10
sema = curio.BoundedSemaphore(MAX_CONNECTIONS_PER_HOST)

async def get_product_info(stock_code):
    url = 'https://shop.countdown.co.nz/Shop/ProductDetails'
    async with sema, curio_http.ClientSession() as session:
         response = await session.get(url, params={'stockcode': stock_code})
         content = await response.text()
         return response, content
        
async def collect_product_info_2(stock_codes, as_df=True):    
    tasks = []
    for code in stock_codes:
        task = await curio.spawn(get_product_info(code))
        tasks.append(task)

    results = []
    for task in tasks:
        response, content = await task.join()
        if response.status_code == 200:
            results.append(parse_product_info(content))
    
    if as_df:
        results = pd.DataFrame(results)
        results['datetime'] = pd.to_datetime(results['datetime'])
    
    return results 

In [26]:
# Test some
codes = [
    '281739',  # cheese
    '260803',  # chocolate
    '701829',  # olive oil
    '381895',  # toilet paper
    '700630',  # Dijon mustard 
    '700631',  # Whole grain mustard
    '360257',  # Pic's peanut butter 380g
    '887052',  # Pic's peanut butter 1000g
    '271818',  # coconut cream
]

%time f = collect_product_info_1(codes)
f

CPU times: user 524 ms, sys: 12 ms, total: 536 ms
Wall time: 3.19 s


,name,description,size,on_sale,sale_price,price,cup_price,datetime
0,Mainland Cheese Block Organic Cheddar,Mainland organic cheddar is a mild cheddar che...,500g,False,NaN,11.00,$22.00/1KG,2017-05-27 17:02:15
1,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,True,3.50,3.89,$3.50/100G,2017-05-27 17:02:15
2,Lupi Olive Oil Organic Extra Virgin,None,750ml,False,NaN,15.00,$2.00/100ML,2017-05-27 17:02:15
3,Earthcare Toilet Paper 6pk Double Lenght Sky ...,None,6pk,False,NaN,5.00,$0.19/100SS,2017-05-27 17:02:16
4,Macro Organic Mustard Dijon,None,200g,True,2.99,3.49,None,2017-05-27 17:02:16
5,Macro Organic Mustard Wholegrain,None,200g,True,2.99,3.49,None,2017-05-27 17:02:16
6,Pics Peanut Butter Crunchy,None,380g,False,NaN,6.50,$1.71/100G,2017-05-27 17:02:17
7,Pics Peanut Butter Crunchy,None,1kg,True,16.50,18.49,$16.50/1KG,2017-05-27 17:02:17
8,Ceres Organics Coconut Cream Creamy & Unsweetened,Creamy and unsweetened,can 400ml,False,NaN,4.25,$1.06/100G,2017-05-27 17:02:18


In [27]:
%time g = curio.run(collect_product_info_2(codes))
g

CPU times: user 436 ms, sys: 0 ns, total: 436 ms
Wall time: 725 ms


,name,description,size,on_sale,sale_price,price,cup_price,datetime
0,Mainland Cheese Block Organic Cheddar,Mainland organic cheddar is a mild cheddar che...,500g,False,NaN,11.00,$22.00/1KG,2017-05-27 17:02:25
1,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,True,3.50,3.89,$3.50/100G,2017-05-27 17:02:25
2,Lupi Olive Oil Organic Extra Virgin,None,750ml,False,NaN,15.00,$2.00/100ML,2017-05-27 17:02:25
3,Earthcare Toilet Paper 6pk Double Lenght Sky ...,None,6pk,False,NaN,5.00,$0.19/100SS,2017-05-27 17:02:25
4,Macro Organic Mustard Dijon,None,200g,True,2.99,3.49,None,2017-05-27 17:02:25
5,Macro Organic Mustard Wholegrain,None,200g,True,2.99,3.49,None,2017-05-27 17:02:25
6,Pics Peanut Butter Crunchy,None,380g,False,NaN,6.50,$1.71/100G,2017-05-27 17:02:25
7,Pics Peanut Butter Crunchy,None,1kg,True,16.50,18.49,$16.50/1KG,2017-05-27 17:02:25
8,Ceres Organics Coconut Cream Creamy & Unsweetened,Creamy and unsweetened,can 400ml,False,NaN,4.25,$1.06/100G,2017-05-27 17:02:25
